In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
from newspaper import Article
import re


#ticker_list = pd.read_csv('ticker_list.txt',header=None)
ticker_list = ['RIO', 'XOM', 'GE', 'F', 'MO', 'XRX', 'GS', 'HBC', 'JPM', 'LYG', 'MS', 'RF']
headers = {'User-Agent':
               'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) '
               'AppleWebKit/537.36 (KHTML, like Gecko)'
               'Chrome/39.0.2171.95 '
               'Safari/537.36'}

for ticker in ticker_list:
    linklist = []
    page = 1
    
    # crawl all transcripts links for a single ticker 
    while True:
        url = 'https://seekingalpha.com/symbol/'+ ticker + '/earnings/more_transcripts?page=' + str(page)
        wb = requests.get(url, headers=headers)
        soup = BeautifulSoup(wb.text, 'lxml')
        if not soup.select('div div a'):
            break
        else:
            for i in soup.select('div div a'):
                linklist.append('https://seekingalpha.com' + i.get('href').strip('\"\\') + '?part=single')
        
        page += 1
        time.sleep(10)
    time.sleep(30)
    
    print('All links got\n')
    
    df = pd.DataFrame(columns=['Ticker', 'Date', 'Time'])
    df_link = pd.DataFrame(columns=['Link', 'Fail_Link'])
    
    for ind, link in enumerate(linklist):
        print(link)
        art = Article(link, language='en', fetch_images=False, memoize_articles=False, browser_user_agent='Mozilla/5.0')
        time.sleep(5)
        art.download()
        
        # test whether downloading succeed and store links to link dataframe
        try:
            art.parse()
            df_link.loc[ind, 'Link'] = link
        except:
            print(link)
            df_link.loc[ind, 'Fail_Link'] = link
            continue
        content = re.split(r'\n+', art.text)
        
        try:
            file_name = ticker + '_' + content[1].replace(' ', '_') + '.txt'
            df.loc[i, 'Ticker'] = ticker
            df.loc[i, 'Date'] = content[1]
            df.loc[i, 'Time'] = content[2]
            
            with open(r'/Users/maxwelllee54/PycharmProjects/Project4Yim/output/'+file_name, 'w+') as f:
                f.write('\n'.join(content))
            
        except IndexError:
            print(content)
            print(i)
            continue 
        
        if not df.empty:
            df.to_csv('time_data.csv', mode='a')
        
        if not df_link.empty:
            df.to_csv('link_data.csv', mode='a')
        
        time.sleep(10)